## Why this for an EDA?

* Real-World Relevance: Analyzing restaurant data can provide actionable insights for businesses and consumers in the area.
* Rich Analysis Potential: You can explore various dimensions such as restaurant types, geographic distribution, price ranges, and socio-economic factors.
* Data Cleaning and Enrichment: Even if the dataset isn't messy, you can enrich it by adding additional data sources, such as income levels, demographic information, or reviews.

In [213]:
import pandas as pd
import numpy as np
import plotly.express as px
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import json
import re
import ast

In [214]:
df = pd.read_excel('wake_county_restaurants.xlsx')

In [215]:
df.head(3)

,OBJECTID,HSISID,Name,Address 1,Address 2,City,State,Postal Code,Phone Number,Restaurant Open Date,Facility Type,PERMITID,X,Y,Geocode Status,geo_shape,geo_point_2d
0,4508952,4092018277,PIZZA HUT #036626,920 NE MAYNARD RD,NaN,CARY,NC,27513,(919) 469-9991,2019-12-27,Restaurant,21944,-78.768384,35.800435,M,"{""coordinates"":[-78.76838683271491,35.80044249...","35.800442496958084, -78.76838683271491"
1,4508959,4092018301,MEMO'S PIZZA,877 E GANNON AVE,SUITE 105,ZEBULON,NC,27597,(919) 646-0064,2019-12-31,Restaurant,21983,-78.294186,35.830163,M,"{""coordinates"":[-78.29418882307587,35.83016966...","35.83016966447205, -78.29418882307587"
2,4508965,4092018308,STARS AND STRIKES,4020 CAPITAL BLVD,SUITE 160,RALEIGH,NC,27604,(678) 965-5707,2020-01-27,Restaurant,21992,-78.579677,35.835510,M,"{""coordinates"":[-78.5796792488391,35.835517340...","35.83551734072338, -78.5796792488391"


In [216]:
df.columns

Index(['OBJECTID', 'HSISID', 'Name', 'Address 1', 'Address 2', 'City', 'State',
       'Postal Code', 'Phone Number', 'Restaurant Open Date', 'Facility Type',
       'PERMITID', 'X', 'Y', 'Geocode Status', 'geo_shape', 'geo_point_2d'],
      dtype='object')

### Description about the fields

| Field Name | Data Type | Purpose | Required |
|------------|-----------|---------|----------|
| OBJECTID | Integer | Unique identifier for each record | No |
| HSISID | Integer | Health Services Information System ID | No |
| Name | String | Name of the restaurant | Yes |
| Address 1 | String | Primary address of the restaurant | Yes |
| Address 2 | String | Additional address information (if any) | Yes |
| City | String | City where the restaurant is located | Yes |
| State | String | State where the restaurant is located | Yes |
| Postal Code | String | ZIP code of the restaurant | Yes |
| Phone Number | String | Contact number of the restaurant | Yes |
| Restaurant Open Date | Date | Date when the restaurant opened | Yes |
| Facility Type | String | Type of food service facility | No |
| PERMITID | String | Permit ID for the restaurant | No |
| X | Float | Longitude coordinate | Yes |
| Y | Float | Latitude coordinate | Yes |
| Geocode Status | String | Status of geocoding process | No |
| geo_shape | JSON | Geographical shape data | No |
| geo_point_2d | String | Geographical point data | No |

Note: Fields marked as "No" in the Required column are dropped in the subsequent data cleaning process.

In [210]:
df['City'] = df['City'].apply(lambda x: x.lower()) #field manipulation
df = df.drop(columns=['OBJECTID', 'HSISID', 'Facility Type', 'PERMITID', 'Geocode Status', 'geo_shape', 'geo_point_2d']) #drop columns
df['Name Clean'] = df['Name'].apply(lambda x: x.split('#')[0].strip()) #clean name (for fast food remove the # notation)

In [211]:
chain_names = list(set(df[df['Name Clean'].duplicated(keep=False)]['Name Clean'])) #get list of chain restaraunts - if a name is duplicated, it is a chain
df['Check Chain'] = np.where(df['Name Clean'].isin(chain_names), 'Chain', 'Not Chain') #make new column with chain / no chain indicator 

In [6]:
value_counts = df['City'].value_counts().reset_index()
value_counts.columns = ['City', 'Number of Restaurants']
px.bar(value_counts, x='City', y='Number of Restaurants')

In [218]:
#List of cities with enough restaurants - remove all other cities
good_cities = value_counts[value_counts['Number of Restaurants'] > 50]['City'].to_list()
df = df[df['City'].isin(good_cities)]

### Data Manipulation Steps

1. **Column Removal and Data Cleaning**:
   - Dropped unnecessary columns: 'OBJECTID', 'HSISID', 'Facility Type', 'PERMITID', 'Geocode Status', 'geo_shape', 'geo_point_2d'.
   - Created a 'Name Clean' column by removing '#' notations from restaurant names.
    - Converted all city names to lowercase and removed any trailing numbers for consistency.

2. **Chain Restaurant Identification**:
   - Created a list of chain restaurant names by identifying duplicated names in the 'Name Clean' column.
   - Added a new column 'Check Chain' to indicate whether each restaurant is a chain or not.

3. **City Filtering**:
   - Created a plot with the number of restaurants in each city
   - Created a list of cities with more than 50 restaurants and filtered the dataset to include only these cities with sufficient data.

4. **Population Data Preparation**:
   - Loaded population data from 'census_population.xlsx'.
   - Renamed the 'Geography' column to 'City' and dropped the 'State' column.
   - Cleaned city names by removing words like 'city', 'village', 'CDP', 'town' and converting to lowercase.

These steps prepare the data for further analysis by removing unnecessary information, standardizing names, identifying chain restaurants, focusing on cities with significant data, and incorporating population information.


# 

-----------


In [249]:
population = pd.read_excel('nc_census_data.xlsx')
population = population.head().rename(columns=population.iloc[0]).iloc[1:]

In [257]:
population.dtypes

City                object
2020 Population    float64
2021 Population    float64
2022 Population    float64
2023 Population    float64
dtype: object

In [250]:
population.head(3)

,Geographic Area,"April 1, 2020\nEstimates Base",2020.0,2021.0,2022.0,2023.0
1,"Aberdeen town, North Carolina",8521,8627.0,9107.0,9315.0,9488.0
2,"Ahoskie town, North Carolina",4892,4872.0,4762.0,4708.0,4651.0
3,"Alamance village, North Carolina",991,991.0,994.0,1009.0,1028.0


In [254]:
population.rename(columns={'Geographic Area': 'City', 2020.0: '2020 Population', 2021.0: '2021 Population', 2022.0: '2022 Population', 2023.0: '2023 Population'}, inplace=True)
population = population.drop(columns='April 1, 2020\nEstimates Base')

In [255]:
def clean_city(city):
    modified_city = city.split(',')[0]
    split_city = modified_city.lower().split(' ')
    remove_words = ['city', 'village', 'CDP', 'town']
    return ' '.join([word for word in split_city if word not in remove_words])

population['City'] = population['City'].apply(clean_city)

In [256]:
# Explanation of population dataset preparation:

# 1. Loading the data:
#    - We loaded the population data from 'nc_census_data.xlsx' using pd.read_excel()
#    - We then renamed the columns using the first row and removed that row from the data

# 2. Renaming and dropping columns:
#    - We renamed 'Geographic Area' to 'City'
#    - We renamed the year columns to more descriptive names (e.g., '2020.0' to '2020 Population')
#    - We dropped the 'April 1, 2020\nEstimates Base' column as it wasn't needed

# 3. Cleaning city names:
#    - We defined a function 'clean_city' to:
#      a) Remove the state name (everything after the comma)
#      b) Convert to lowercase
#      c) Remove words like 'city', 'village', 'CDP', 'town'
#    - We applied this function to the 'City' column

# 4. Data type conversion and sorting:
#    - We're converting population columns to numeric type
#    - We're sorting the dataframe by the most recent population (2023) in descending order


,City,2020 Population,2021 Population,2022 Population,2023 Population
1,aberdeen,8627.0,9107.0,9315.0,9488.0
2,ahoskie,4872.0,4762.0,4708.0,4651.0
3,alamance,991.0,994.0,1009.0,1028.0
4,albemarle,16379.0,16414.0,16652.0,16936.0


## Join value_counts and population

In [11]:
consolidated_df = pd.merge(value_counts, population, on='City', how='inner')
consolidated_df['Ratio'] = consolidated_df['Population'] / consolidated_df['Number of Restaurants'] #new column that indicates presence of rest vs people
consolidated_df = consolidated_df[consolidated_df['City'].isin(good_cities)]
consolidated_df.sort_values(by='Population', ascending=False)

,City,Number of Restaurants,Most Current Population Year,Population,Ratio
0,raleigh,1230,2023,482295,392.109756
1,cary,430,2023,180010,418.627907
4,apex,120,2023,72225,601.875000
3,wake forest,148,2023,54337,367.141892
7,holly springs,83,2023,46271,557.481928
6,fuquay-varina,91,2023,43817,481.505495
5,garner,97,2023,35265,363.556701
2,morrisville,173,2023,31703,183.254335
8,knightdale,57,2023,20275,355.701754
9,zebulon,36,2023,9401,261.138889


#Create a stacked bar chart to view split between chain and not chain restaraunts 

In [12]:
crosstab = pd.crosstab(df['City'], df['Check Chain'], normalize='index') # Convert to percentages
crosstab *= 100
crosstab = crosstab.round(2).reset_index()
crosstab_melt = crosstab.melt(id_vars='City', var_name='Restaurant Type', value_name='Percentage')

#Plotly express stacked bar chart\
px.bar(crosstab_melt, x='City', y='Percentage', color='Restaurant Type')

In [13]:
crosstab.sort_values(by='Chain', ascending=False)

Check Chain,City,Chain,Not Chain
4,holly springs,48.19,51.81
3,garner,47.42,52.58
8,wake forest,44.59,55.41
5,knightdale,42.11,57.89
0,apex,39.17,60.83
2,fuquay-varina,37.36,62.64
1,cary,34.19,65.81
7,raleigh,33.98,66.02
9,zebulon,33.33,66.67
6,morrisville,24.28,75.72


In [14]:
#View the different types of fast food places and the most popular ones in Wake County
df1 = df[df['Check Chain']=='Chain']
fast_food = df1['Name Clean'].value_counts()
fast_food_df = pd.DataFrame(fast_food)
fast_food_df.head(20)

,count
Name Clean,
SUBWAY,50
STARBUCKS,31
MCDONALD'S,27
WENDY'S,26
CHICK-FIL-A,23
BOJANGLES,23
JERSEY MIKE'S,22
SHEETZ,22
BURGER KING,20


In [15]:
fig = px.scatter_mapbox(
    df,
    lat="Y",
    lon="X",
    hover_name="Name",
    hover_data=["Address 1", "City"],
    color="City",
    zoom=10,
    height=800
)

fig.update_layout(
    mapbox_style="open-street-map",
    title="Restaurant Locations in Wake County, NC"
)

fig.show()
# See if I can find all the common shopping complexs and plot those around areas to showcase the density is based on the shopping areas

In [101]:
#Create a cool plot of all towns with the X & Y coordinates
#Create subplots of each city and have different colored dots for Fast Food vs others

In [ ]:
#Use the Yelp API to get the type of restaunts, the average star review and any menu items

In [16]:
url = 'https://api.yelp.com/v3/businesses/search/phone'
key = 'aXDMy4mi33fsq_SKw6A50fneNIzWxA90PVWaqL9a-7DFVPEYHSD1jjopuufqcQ-dJogcLYCWxkWkqPdLltrY0kWCrzHlWwvd_LrdMQvYjFfC0CYpucmrb7S5MdeRZnYx'
headers = {'Authorization': 'Bearer %s' % key}

In [17]:
def process_phone_number(phone_number): #function used to convert phone number into format required for yelp api 
    if type(phone_number) == str:
        output = phone_number.replace('(919)', '+1919').replace('(984)', '+1984').replace('-', '').replace(' ', '').replace('ext.5', '')
        return output
    
df['Phone Number'] = df['Phone Number'].apply(process_phone_number)

In [18]:
def get_details(phone_number):
    if type(phone_number) == str:
        parameters = {'phone': phone_number}
        response = requests.get(url=url, headers=headers, params=parameters)
        return response.json()

In [192]:
#Run sat Jul 13
yelp_1 = pd.read_excel('Yelp_Batch_1.xlsx')
yelp_2 = pd.read_excel('Yelp_Batch_2.xlsx')
yelp_3 = pd.read_excel('Yelp_Batch_3.xlsx')
yelp_4 = pd.read_excel('Yelp_Batch_4.xlsx')[:522]
final_df = pd.concat([yelp_1, yelp_2, yelp_3, yelp_4])
final_df = final_df.drop(columns=['Name', 'Name Clean'])

In [193]:
final_df

,Unnamed: 0,Address 1,Address 2,City,State,Postal Code,Phone Number,Restaurant Open Date,X,Y,Check Chain,Yelp Details
0,0,920 NE MAYNARD RD,NaN,cary,NC,27513,+19194699991,2019-12-27,-78.768384,35.800435,Chain,{'businesses': [{'id': 'XrWXY_ajpR7nJAGzr3g5JQ...
1,1,877 E GANNON AVE,SUITE 105,zebulon,NC,27597,+19196460064,2019-12-31,-78.294186,35.830163,Not Chain,{'businesses': [{'id': 'pAVqVeJvAEJ8_noz18Whsg...
2,2,4020 CAPITAL BLVD,SUITE 160,raleigh,NC,27604,(678)9655707,2020-01-27,-78.579677,35.835510,Not Chain,"{'businesses': [], 'total': 0}"
3,3,1353 KILDAIRE FARM RD,NaN,cary,NC,27511,+19198001802,2020-01-27,-78.781938,35.759765,Not Chain,{'businesses': [{'id': '9vGngnH26Og_MjILOiGGuw...
4,4,3913 WESTERN BLVD,NaN,raleigh,NC,27606,+19196131286,2020-02-07,-78.691069,35.784021,Not Chain,"{'businesses': [], 'total': 0}"
...,...,...,...,...,...,...,...,...,...,...,...,...
517,2060,1430 W WILLIAMS ST,NaN,apex,NC,27523-9300,+19193031006,2021-11-05,-78.876574,35.752456,Not Chain,{'businesses': [{'id': 'LjvHGYAC113ZGUs0mue43w...
518,2061,100 MACALYSON CT,NaN,cary,NC,27511,+19194674444,2021-11-15,-78.800617,35.735069,Not Chain,{'businesses': [{'id': '9QrZvEVV_eXBZ-2AuhZNuA...
519,2062,3701 HILLSBOROUGH ST,NaN,raleigh,NC,27614,+19196008708,2021-11-05,-78.685657,35.793542,Not Chain,"{'businesses': [], 'total': 0}"
520,2063,2920 SHERMAN OAK PL,STE 140,raleigh,NC,27609,+19842723300,2021-10-26,-78.620829,35.821317,Chain,{'businesses': [{'id': 'IPEvg78LskbnWg9sCOmcvg...


In [201]:
df[2023:]
# df_4['Yelp Details'] = df_4['Phone Number'].apply(get_details)
# #df_5['Yelp Details'] = df_5['Phone Number'].apply(get_details)
# df_4.to_excel('Yelp_Batch_4.xlsx')
#df_5.to_excel('Yelp_Batch_5.xlsx')       

,Name,Address 1,Address 2,City,State,Postal Code,Phone Number,Restaurant Open Date,X,Y,Name Clean,Check Chain
2066,CUCCIOLO TERRAZZA,4200 SIX FORKS RD,SUITE 100,raleigh,NC,26802-7609,+19197477887,2021-11-23,-78.638965,35.835923,CUCCIOLO TERRAZZA,Not Chain
2067,AVENT FERRY PIZZA,3231 AVENT FERRY RD,SUITE 102,raleigh,NC,27606-3171,+19197030027,2021-12-02,-78.695655,35.766740,AVENT FERRY PIZZA,Not Chain
2069,CHIPOTLE,601 VILLAGE WALK DR,NaN,holly springs,NC,27540,+19195868560,2021-12-08,-78.835930,35.639830,CHIPOTLE,Chain
2070,THE GARDEN GRILL & BAR,1500 RDU CENTER DR,NaN,morrisville,NC,27560-8278,+19198488088,2021-12-13,-78.796045,35.851069,THE GARDEN GRILL & BAR,Not Chain
2071,BUL BOX,2526 HILLSBOROUGH ST,STE 100,raleigh,NC,27607,+19198805900,2021-12-28,-78.669227,35.788349,BUL BOX,Chain
...,...,...,...,...,...,...,...,...,...,...,...,...
2507,PNC CLUB KITCHEN,1400 EDWARDS MILL RD,NaN,raleigh,NC,27607-3624,+19198612300,1999-10-28,-78.722939,35.803245,PNC CLUB KITCHEN,Not Chain
2508,PNC 118-120,1400 EDWARDS MILL RD,NaN,raleigh,NC,27607-3624,+19198612300,1999-12-07,-78.722939,35.803245,PNC 118-120,Not Chain
2509,KADHAI-THE INDIAN WOK,6260 GLENWOOD AVE,STE 12,raleigh,NC,27612-2650,+19197859730,1992-07-29,-78.703135,35.858765,KADHAI-THE INDIAN WOK,Not Chain
2510,ARENA CAFE- CISCO BLDG 7,7100 KIT CREEK RD,NaN,morrisville,NC,27560,+19196996109,2000-04-01,-78.875530,35.857848,ARENA CAFE- CISCO BLDG 7,Not Chain


In [175]:
yelp_4['Filter From'] = yelp_4['Yelp Details'].apply(lambda x: 'bad' if isinstance(x, dict) and 'error' in x else 'good' if isinstance(x, dict) else 'invalid')

In [191]:
# View the last 50 rows of yelp_4
yelp_4[:522]


,Unnamed: 0,Name,Address 1,Address 2,City,State,Postal Code,Phone Number,Restaurant Open Date,X,Y,Name Clean,Check Chain,Yelp Details,Filter From
0,1529,COOK OUT #297,1151 N SMITHFIELD RD,NaN,knightdale,NC,27545,+19196188532,2021-12-29,-78.488102,35.797845,COOK OUT,Chain,"{'businesses': [], 'total': 0}",invalid
1,1530,PINEAPPLE SOL,302 COLONADES WAY,STE 109,cary,NC,27518,+19198038533,2022-01-04,-78.777565,35.738534,PINEAPPLE SOL,Not Chain,"{'businesses': [], 'total': 0}",invalid
2,1531,V PIZZA,7930 SKYLAND RIDGE PKY,STE 100,raleigh,NC,27617,+19197483633,2021-12-06,-78.783800,35.915834,V PIZZA,Chain,{'businesses': [{'id': 'z4J6T88PypSvJo7VgDWSBQ...,invalid
3,1532,ARBY'S #76036,151 WAKELON ST,NaN,zebulon,NC,27597,+19192690995,2022-01-19,-78.315729,35.833475,ARBY'S,Chain,{'businesses': [{'id': 'CNpVd0dF9gs7X76suQ5mdA...,invalid
4,1533,JUBALA COFFEE,200 PARK AT NORTH HILLS ST,SUITE 140,raleigh,NC,27609,+19192005048,2022-01-24,-78.637533,35.835346,JUBALA COFFEE,Chain,"{'businesses': [], 'total': 0}",invalid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,2060,DANIEL'S RESTAURANT,1430 W WILLIAMS ST,NaN,apex,NC,27523-9300,+19193031006,2021-11-05,-78.876574,35.752456,DANIEL'S RESTAURANT,Not Chain,{'businesses': [{'id': 'LjvHGYAC113ZGUs0mue43w...,invalid
518,2061,HOMEWOOD SUITES RALEIGH/CARY FOOD SERVICE,100 MACALYSON CT,NaN,cary,NC,27511,+19194674444,2021-11-15,-78.800617,35.735069,HOMEWOOD SUITES RALEIGH/CARY FOOD SERVICE,Not Chain,{'businesses': [{'id': '9QrZvEVV_eXBZ-2AuhZNuA...,invalid
519,2062,GYM TACOS,3701 HILLSBOROUGH ST,NaN,raleigh,NC,27614,+19196008708,2021-11-05,-78.685657,35.793542,GYM TACOS,Not Chain,"{'businesses': [], 'total': 0}",invalid
520,2063,BB'S CRISPY CHICKEN,2920 SHERMAN OAK PL,STE 140,raleigh,NC,27609,+19842723300,2021-10-26,-78.620829,35.821317,BB'S CRISPY CHICKEN,Chain,{'businesses': [{'id': 'IPEvg78LskbnWg9sCOmcvg...,invalid


In [80]:
# Function to clean and load JSON strings
def clean_and_load_json(value):
    if pd.isna(value):
        return None
    try:
        return ast.literal_eval(value)
    except (ValueError, SyntaxError) as e:
        print(f"Error evaluating string: {e}")
        return None

def select_only_open_restaurants(yelp_details):
    if isinstance(yelp_details, dict) and 'total' in yelp_details:
        for item in yelp_details['businesses']:
            if item['is_closed'] == False:
                return item
    return None

def process_yelp_details(value):
    # Clean and load JSON
    cleaned_json = clean_and_load_json(value)
    if cleaned_json is None:
        return None
    
    # Select only open restaurants
    return select_only_open_restaurants(cleaned_json)

# # Apply the function to the column
final_df['Yelp Details Clean'] = final_df['Yelp Details'].apply(process_yelp_details)


# # # Apply the function to the column
# final_df['Yelp Details'] = final_df['Yelp Details'].apply(clean_and_load_json)

final_df.head(6)


,Unnamed: 0,Address 1,Address 2,City,State,Postal Code,Phone Number,Restaurant Open Date,X,Y,Check Chain,Yelp Details,Yelp Details Clean
0,0,920 NE MAYNARD RD,NaN,cary,NC,27513,+19194699991,2019-12-27,-78.768384,35.800435,Chain,{'businesses': [{'id': 'XrWXY_ajpR7nJAGzr3g5JQ...,"{'id': 'XrWXY_ajpR7nJAGzr3g5JQ', 'alias': 'piz..."
1,1,877 E GANNON AVE,SUITE 105,zebulon,NC,27597,+19196460064,2019-12-31,-78.294186,35.830163,Not Chain,{'businesses': [{'id': 'pAVqVeJvAEJ8_noz18Whsg...,"{'id': 'pAVqVeJvAEJ8_noz18Whsg', 'alias': 'mem..."
2,2,4020 CAPITAL BLVD,SUITE 160,raleigh,NC,27604,(678)9655707,2020-01-27,-78.579677,35.835510,Not Chain,"{'businesses': [], 'total': 0}",None
3,3,1353 KILDAIRE FARM RD,NaN,cary,NC,27511,+19198001802,2020-01-27,-78.781938,35.759765,Not Chain,{'businesses': [{'id': '9vGngnH26Og_MjILOiGGuw...,"{'id': '9vGngnH26Og_MjILOiGGuw', 'alias': 'sze..."
4,4,3913 WESTERN BLVD,NaN,raleigh,NC,27606,+19196131286,2020-02-07,-78.691069,35.784021,Not Chain,"{'businesses': [], 'total': 0}",None
5,5,101 DICKENS RD,NaN,fuquay-varina,NC,27526,+19192449788,2020-03-02,-78.811333,35.615946,Not Chain,"{'businesses': [], 'total': 0}",None


In [88]:
def extract_detail(field, field_name):
    try:
        return field[field_name]
        # if isinstance(field, dict): # and 'businesses' in field:
        #     return field['businesses'][0][field_name]
    except:
        return 'Field does not exist'
    
def create_new_columns(df):
    columns = ['name', 'is_closed', 'url', 'review_count', 'rating', 'review_count', 'categories', 'price']
    for col in columns:
        df[col] = df['Yelp Details Clean'].apply(lambda x: extract_detail(x, col))
    
    return df
    

In [89]:
final_df = create_new_columns(final_df)

In [91]:
def parse_category(category):
    if isinstance(category, list):
        return [val['title'] for val in category]
    else:
        return 'Field does not exist'

In [92]:
final_df['categories'] = final_df['categories'].apply(parse_category)

#Explain everything done up until here

#Fun Analysis
- Get counts for how many I found yelp matches for 
- Get the categories and find the top most prevelant categories in the area and then in each of the 10 cities
- Pull the top 50 highest rated restaurants (See if any are fast food lol) --> then find the top 20 highest rated fast food places
- Do some analysis on relationship between review count and overall review #

In [93]:
final_df.head(3)

,Unnamed: 0,Address 1,Address 2,City,State,Postal Code,Phone Number,Restaurant Open Date,X,Y,Check Chain,Yelp Details,Yelp Details Clean,name,is_closed,url,review_count,rating,categories,price
0,0,920 NE MAYNARD RD,NaN,cary,NC,27513,+19194699991,2019-12-27,-78.768384,35.800435,Chain,{'businesses': [{'id': 'XrWXY_ajpR7nJAGzr3g5JQ...,"{'id': 'XrWXY_ajpR7nJAGzr3g5JQ', 'alias': 'piz...",Pizza Hut,False,https://www.yelp.com/biz/pizza-hut-cary-3?adju...,35,2.0,"[Pizza, Chicken Wings, Fast Food]",$
1,1,877 E GANNON AVE,SUITE 105,zebulon,NC,27597,+19196460064,2019-12-31,-78.294186,35.830163,Not Chain,{'businesses': [{'id': 'pAVqVeJvAEJ8_noz18Whsg...,"{'id': 'pAVqVeJvAEJ8_noz18Whsg', 'alias': 'mem...",Memo’s Pizza,False,https://www.yelp.com/biz/memo-s-pizza-zebulon?...,16,4.2,"[Italian, Seafood, Sandwiches]",Field does not exist
2,2,4020 CAPITAL BLVD,SUITE 160,raleigh,NC,27604,(678)9655707,2020-01-27,-78.579677,35.835510,Not Chain,"{'businesses': [], 'total': 0}",None,Field does not exist,Field does not exist,Field does not exist,Field does not exist,Field does not exist,Field does not exist,Field does not exist


In [448]:
from collections import Counter
list_categories = final_df['categories'].tolist()
flat_list_cats = Counter([cat for cats in list_categories for cat in cats if cats != 'Field does not exist'])
flat_list_cats

Counter({'Fast Food': 129,
         'Sandwiches': 101,
         'Burgers': 72,
         'Pizza': 66,
         'Coffee & Tea': 66,
         'Breakfast & Brunch': 61,
         'American': 61,
         'Mexican': 54,
         'Chinese': 48,
         'Salad': 45,
         'Italian': 41,
         'Seafood': 41,
         'Chicken Wings': 38,
         'Japanese': 26,
         'New American': 25,
         'Bars': 23,
         'Sushi Bars': 23,
         'Delis': 22,
         'Juice Bars & Smoothies': 20,
         'Hotels': 20,
         'Wine Bars': 19,
         'Soup': 19,
         'Indian': 19,
         'Grocery': 18,
         'Southern': 18,
         'Mediterranean': 18,
         'Sports Bars': 18,
         'Cocktail Bars': 17,
         'Steakhouses': 17,
         'Chicken Shop': 16,
         'Desserts': 16,
         'Hot Dogs': 15,
         'Barbeque': 15,
         'Convenience Stores': 15,
         'Tex-Mex': 14,
         'Gas Stations': 14,
         'Venues & Event Spaces': 13,
         'B

In [94]:
final_df['price'].value_counts()

price
Field does not exist    411
$$                      308
$                       263
$$$                      16
$$$$                      2
Name: count, dtype: int64

In [95]:
final_df[(final_df['price'] != '$') & (final_df['price'] != '$$') & (final_df['price'] != 'Field does not exist') & (final_df['price'].notna())]['City'].value_counts()

City
raleigh    14
cary        3
garner      1
Name: count, dtype: int64

In [96]:
final_df[(final_df['price'] != '$') & (final_df['price'] != '$$') & (final_df['price'] != 'Field does not exist') & (final_df['price'].notna())]['name']

70        Second Empire Restaurant And Tavern
71                           Rey's Restaurant
106     Fleming’s Prime Steakhouse & Wine Bar
181     Renaissance Raleigh North Hills Hotel
205                               Lowes Foods
276                     Sullivan's Steakhouse
362                        Barcelona Wine Bar
387                  Ruth's Chris Steak House
405    42nd Street Oyster Bar & Seafood Grill
422                      Margaux's Restaurant
427                  Ruth's Chris Steak House
437            Maximillians Grill & Wine Bar 
470     Perry's Steakhouse & Grille - Raleigh
57                                    Poole's
106                                  Mandolin
109                                China Cary
194                           Galaxy Fun Park
197                            Amorino Gelato
Name: name, dtype: object

In [97]:
rating_df = final_df[final_df['Yelp Details'].notna() & (final_df['review_count'] != 'Field does not exist') & (final_df['rating'] != 'Field does not exist')]

In [101]:
# Scatter plot of rating vs review count, colored by Check Chain
fig_1 = px.scatter(rating_df, x='review_count', y='rating', color='Check Chain', 
                   title='Rating vs Review Count by Chain Status',
                   labels={'review_count': 'Number of Reviews', 'rating': 'Rating'})

# Box plot of ratings by Check Chain
fig_2 = px.box(rating_df, x='Check Chain', y='rating', 
                title='Distribution of Ratings by Chain Status',
                labels={'rating': 'Rating'})

# Bar plot of top 10 categories
from collections import Counter
list_categories = final_df['categories'].tolist()
flat_list_cats = Counter([cat for cats in list_categories for cat in cats if cats != 'Field does not exist'])
top_10_categories = pd.DataFrame(flat_list_cats.most_common(10), columns=['Category', 'Count'])
fig_3 = px.bar(top_10_categories, x='Category', y='Count', 
               title='Top 10 Restaurant Categories',
               labels={'Count': 'Number of Restaurants'})

# Pie chart of price ranges
price_counts = final_df['price'].value_counts()
fig_4 = px.pie(values=price_counts.values, names=price_counts.index, 
               title='Distribution of Price Ranges',
               labels={'label': 'Price Range', 'value': 'Count'})

# Display all plots
fig_1.show()
fig_2.show()
fig_3.show()
fig_4.show()

In [99]:
rating_df.groupby('Check Chain').rating.agg('mean')

Check Chain
Chain        2.958521
Not Chain    3.761174
Name: rating, dtype: object

In [123]:
# Create a pivot table to show the distribution of chain vs not chain restaurants in each city
pivot_table = pd.pivot_table(final_df, values='Unnamed: 0', index='City', columns='Check Chain', aggfunc='count', fill_value=0)

# Calculate percentages
pivot_table_percentage = pivot_table.div(pivot_table.sum(axis=1), axis=0) * 100

# Sort the table by the total number of restaurants (descending)
pivot_table_sorted = pivot_table_percentage.sort_values(by='Chain', ascending=False)

# Display the pivot table
print("Distribution of Chain vs Non-Chain Restaurants by City (%):")
print(pivot_table_sorted.round(2))

# Visualize the distribution using a stacked bar chart
fig = px.bar(pivot_table_sorted, 
             title='Distribution of Chain vs Non-Chain Restaurants by City',
             labels={'value': 'Percentage', 'City': 'City', 'Check Chain': 'Restaurant Type'},
             height=600)

fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.show()

# Create a geographic plot with cities colored based on chain percentage
import plotly.express as px

# Calculate the percentage of non-chain restaurants for each city
city_percentages = pivot_table_percentage['Not Chain'].sort_values(ascending=False)

# Create a DataFrame with city coordinates and percentages
city_data = final_df.groupby('City').first().reset_index()
city_data = city_data.merge(city_percentages, left_on='City', right_index=True)

# Create the scatter plot
fig = px.scatter_mapbox(city_data, 
                        lat='Y', 
                        lon='X', 
                        color='Not Chain',
                        size='Not Chain',
                        hover_name='City',
                        hover_data=['Not Chain'],
                        color_continuous_scale=px.colors.sequential.Agsunset,
                        zoom=9,
                        mapbox_style="open-street-map",
                        title='Distribution of Non-Chain Restaurants by City')

# Customize the plot
fig.update_layout(
    title_font_size=16,
    coloraxis_colorbar=dict(title='Percentage of Non-Chain Restaurants'),
    height=800,
    width=1200
)

# Add text labels for cities
fig.add_trace(px.scatter_mapbox(city_data, 
                                lat='Y', 
                                lon='X', 
                                text='City',
                                zoom=9,
                                mapbox_style="open-street-map").data[0])

fig.update_traces(textposition="top center", selector=dict(type='scattermapbox'))

# Set the map center and zoom level
center_lat = city_data['Y'].mean()
center_lon = city_data['X'].mean()
fig.update_layout(mapbox=dict(center=dict(lat=center_lat, lon=center_lon), zoom=9))

fig.show()


Distribution of Chain vs Non-Chain Restaurants by City (%):
Check Chain    Chain  Not Chain
City                           
garner         48.57      51.43
wake forest    48.44      51.56
holly springs  43.75      56.25
apex           42.00      58.00
fuquay-varina  37.14      62.86
raleigh        36.93      63.07
cary           33.93      66.07
knightdale     29.63      70.37
morrisville    23.29      76.71
zebulon        20.00      80.00


In [162]:
chain_df = final_df[final_df['Check Chain'] == 'Chain']
fast_food_counts = {}
fast_food_reviews = {}
fast_food_review_counts = {}

for idx, row in chain_df.iterrows():
    name = row['name']
    if name != 'Field does not exist':
        fast_food_counts[name] = fast_food_counts.get(name, 0) + 1
        
        if 'rating' in row and not pd.isna(row['rating']):
            fast_food_reviews[name] = fast_food_reviews.get(name, 0) + row['rating']
            fast_food_review_counts[name] = fast_food_review_counts.get(name, 0) + 1

fast_food_df = pd.DataFrame(list(fast_food_counts.items()), columns=['Restaurant', 'Count'])
fast_food_df_reviews = pd.DataFrame(list(fast_food_reviews.items()), columns=['Restaurant', 'Review Sum'])
fast_food_df_review_counts = pd.DataFrame(list(fast_food_review_counts.items()), columns=['Restaurant', 'Review Count'])

# Merge the dataframes
merged_df = fast_food_df.merge(fast_food_df_reviews, on='Restaurant', how='left')
merged_df = merged_df.merge(fast_food_df_review_counts, on='Restaurant', how='left')

# Calculate average rating
merged_df['Average Rating'] = merged_df['Review Sum'] / merged_df['Count']

# Sort by Average Rating
merged_df = merged_df.sort_values(by='Average Rating', ascending=False)

merged_df
# Convert 'Review Sum' to numeric, replacing NaN with 0
# fast_food_df['Average Rating'] = fast_food_df['Review Sum'] / fast_food_df['Count']
# fast_food_df.sort_values(by='Average Rating', ascending=False)
# fast_food_df.head(20)

,Restaurant,Count,Review Sum,Review Count,Average Rating
82,Feng Cha Raleigh,1,5.0,1,5.000000
65,Karavan by 3 Olives,1,4.7,1,4.700000
139,Kwench Juice Cafe,1,4.5,1,4.500000
56,Aladdin's Eatery - North Hills,1,4.5,1,4.500000
123,Taza Grill,1,4.5,1,4.500000
...,...,...,...,...,...
9,Wendy's,8,15.0,8,1.875000
86,Burger King,4,7.3,4,1.825000
41,McDonald's,18,32.0,18,1.777778
0,Pizza Hut,3,5.2,3,1.733333


In [164]:
merged_df[merged_df['Review Count'] > 5].sort_values(by='Average Rating', ascending=False)

,Restaurant,Count,Review Sum,Review Count,Average Rating
39,Sheetz,8,26.2,8,3.275000
3,Starbucks,13,39.9,13,3.069231
51,Chick-fil-A,8,23.4,8,2.925000
64,Jersey Mike's Subs,7,19.7,7,2.814286
14,Panera Bread,7,18.5,7,2.642857
92,Tropical Smoothie Cafe,6,14.7,6,2.450000
4,Subway,20,48.9,20,2.445000
20,Taco Bell,8,16.6,8,2.075000
8,Dunkin',6,11.5,6,1.916667
9,Wendy's,8,15.0,8,1.875000
